In [9]:
import os

In [10]:
import os
import sys
sys.path.insert(0, "C:\\Users\\KonuTech\\DataSpellProjects\\kaggle-tabular-playground-series-dec-2021\\scripts")

In [11]:
from scripts.eda import *

In [19]:
from scripts.unzip import get_unzip

In [27]:
get_unzip('tabular-playground-series-dec-2021.zip')

FileNotFoundError: [Errno 2] No such file or directory: 't'

In [14]:
os.getcwd()

'C:\\Users\\KonuTech\\DataSpellProjects\\kaggle-tabular-playground-series-dec-2021'

In [16]:
INPUTS = os.getcwd() + "\\INPUTS"
SCRIPTS = os.getcwd() + "\\SCRIPTS"